In [2]:
import pandas as pd 
import numpy as np
execfile("../toolbox.py")

In [3]:
variables = ["gini_net_SWIID", "years_schooling", "GDP_PC_WB", "natural_ressources_WB"]

data = pd.read_csv("../data_source/all_data.csv", usecols=variables + ['code', 'year'], index_col=['code', 'year'])

data.reset_index(inplace=True)
data.sort_values(['code', 'year'], inplace=True)
data.set_index(['code'], inplace=True)
new_frame = pd.DataFrame()

# extrapolating years of schooling to yearly data
for country in set(data.index.values):
    sel = data.loc[country]
    if sel.shape != (len(variables) + 1,):
        try:
            sel.loc[:, 'years_schooling'] = sel['years_schooling'].interpolate()
        except TypeError:
            None
        new_frame = pd.concat([new_frame, sel])
data = new_frame.reset_index().set_index(['code', 'year']).sort_index(level=[0,1]).dropna(how='all')

# adding a economic shock data
# data['shock'] = 0
# data.loc[(data.index.levels[0].tolist(), 1974), 'shock'] = 1
# data.loc[(data.index.levels[0].tolist(), 1981), 'shock'] = 1
# data.loc[(data.index.levels[0].tolist(), 1982), 'shock'] = 1
# data.loc[(data.index.levels[0].tolist(), 2008), 'shock'] = 1
# data.loc[(data.index.levels[0].tolist(), 2009), 'shock'] = 1
print data.keys()

# computing growth
data['growth'] = growth(data, 'GDP_PC_WB', how='past', as_rate=True)

# renaming the columns
data.columns = ["gdp", "nat_re","schl", "gini", "g"]

data_ten_y = resample(data, 10, 5).dropna()
data_ten_y.info()

print "Number of countries ", len(data_ten_y.index.levels[0])

Index([u'GDP_PC_WB', u'natural_ressources_WB', u'years_schooling',
       u'gini_net_SWIID'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 325 entries, (ALB, 2005.0) to (ZWE, 2005.0)
Data columns (total 5 columns):
gdp       325 non-null float64
nat_re    325 non-null float64
schl      325 non-null float64
gini      325 non-null float64
g         325 non-null float64
dtypes: float64(5)
memory usage: 15.2+ KB
Number of countries  210


In [7]:
var_1 = 'gini'
var_2 = 'gdp'
output = 'g'
functions = ['mean', 'size']
nbr_quartiles = 2
quart = quartilize_periodwise(data_ten_y, nbr_quartiles)
quart.columns = ['qu_' + var for var in quart.keys()]
frame = pd.concat([data_ten_y, quart], axis=1)
center = frame.groupby(['qu_' + var_1, 'qu_' + var_2]).agg(functions)[output].stack().unstack('qu_' + var_1)
last_col = frame.groupby(['qu_' + var_2]).agg(functions)[output].stack()
last_col.columns = 'all'
last_row = frame.groupby(['qu_' + var_1]).agg(functions)[output].transpose()
last_row['0'] = [''] * len(functions)
last_row.index = pd.MultiIndex.from_tuples([("all", var) for var in last_row.index])
full_mat = pd.concat([pd.concat([center, last_col], axis=1), 
                      last_row])
full_mat.columns = range(nbr_quartiles) + ['all']
full_mat.index.names = [var_2, var_1]
full_mat

0           1        all
gdp gini                                   
0   mean    0.040670    0.042033  0.0415841
    size   53.000000  108.000000        161
1   mean    0.063325    0.051195   0.059183
    size  108.000000   56.000000        164
all mean    0.055867    0.045161           
    size  161.000000  164.000000

In [17]:
full_mat[0] - full_mat[1]

gdp  gini
0    mean    -0.001363
     size   -55.000000
1    mean     0.012130
     size    52.000000
all  mean     0.010706
     size    -3.000000
dtype: float64

In [9]:
last_row.keys()

Int64Index([0, 1, 0], dtype='int64', name=u'qu_gini')

In [10]:
('qu_' + var_1, i) for i in (range(nbr_quartiles) + ['all'])

SyntaxError: invalid syntax (<ipython-input-10-ea450c7f52e7>, line 1)